### RandomeForest with Pipeline and hyperparameter tunning

In [4]:
import pandas as pd
import seaborn as sns

In [5]:
df = sns.load_dataset('tips')

In [6]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [7]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [8]:
X = df.drop('time',axis = 1)

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
encoder = LabelEncoder()

In [11]:
df['time']= encoder.fit_transform(df['time'])

In [12]:
y= df.time

In [13]:
y.unique()

array([0, 1])

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.20,random_state=42)

In [16]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((195, 6), (49, 6), (195,), (49,))

### Pipleine Fitting

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [18]:
num_categorical_data = ['total_bill','tip','size']
obj_categorical_data = ['sex','smoker','day']

In [19]:
num_pipeline = Pipeline(steps = [
    ('imputer',SimpleImputer(strategy='median')),#handling missing values
    ('scaler',StandardScaler())#feature scaling
])


obj_pipeline = Pipeline(steps = [
    ('imputer',SimpleImputer(strategy='most_frequent')),#handling missing values
    ('encoder',OneHotEncoder()) # categorical feature to numerical feature
])

In [20]:
processor = ColumnTransformer(
    [
        ('num_categorical_data',num_pipeline,num_categorical_data),
        ('obj_categorical_data',obj_pipeline,obj_categorical_data)
    ]
)

In [21]:
X_train = processor.fit_transform(X_train)

In [22]:
X_test = processor.fit_transform(X_test)

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [24]:
model = {
    'Random Forest' : RandomForestClassifier(),
    'LogisticRegression' : LogisticRegression(),
    'DecisionTree' : DecisionTreeClassifier()
}

In [25]:
for i in range(len(model)):
    print(list(model.values())[i])

RandomForestClassifier()
LogisticRegression()
DecisionTreeClassifier()


In [26]:
def report_generator(X_train,X_test,y_train,y_test,model):
    report = {}
    for i in range(len(model)):
        Predictmodel = list(model.values())[i]
        Predictmodel.fit(X_train,y_train)
        y_predict = Predictmodel.predict(X_test)

        test_model_score = accuracy_score(y_test,y_predict)
        report[list(model.keys())[i]] = test_model_score
    return report
        
        

In [27]:
report_generator(X_train,X_test,y_train,y_test,model)

{'Random Forest': 0.9795918367346939,
 'LogisticRegression': 1.0,
 'DecisionTree': 0.9795918367346939}

### HyperParameter tunning with randomized searchCV

In [28]:
from sklearn.model_selection import RandomizedSearchCV

In [29]:
parameter = {
    'n_estimators':[100,200,300,400,],
    'criterion':['gini','entropy','log_loss'],
    'max_depth':[100,200,300,400,None],
    
}

In [32]:
cv = RandomizedSearchCV(RandomForestClassifier(),param_distributions=parameter,cv=5,verbose=3,scoring='accuracy',random_state=42)

In [33]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=100, n_estimators=100;, score=0.974 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=100, n_estimators=100;, score=0.923 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=100, n_estimators=100;, score=0.974 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=100, n_estimators=100;, score=0.923 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=100, n_estimators=100;, score=0.949 total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=200, n_estimators=200;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=200, n_estimators=200;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=200, n_estimators=200;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=200, n_estimators=200;, score=0.949 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=200, n_estimators=200;, score=0.949 total time=   0.1s

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [100, 200, 300, 400, None],
                                        'n_estimators': [100, 200, 300, 400]},
                   random_state=42, scoring='accuracy', verbose=3)

In [34]:
cv.best_estimator_

RandomForestClassifier(max_depth=200, n_estimators=200)

In [36]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 200, 'criterion': 'gini'}

In [37]:
cv_y_pred = cv.predict(X_test)

In [38]:
accuracy_score(y_test,cv_y_pred)

0.9795918367346939